# Raspagem de dados do site da Câmara dos deputados
<p> Web scraper do site da Câmara dos deputados(CdB) utilizando a biblioteca selenium para scraping, pandas para formatação e time para testes temporizados. A iniciativa nasceu da dificuldade de baixar chaves de pesquisas com muitas proposições, o site do CdB limita que somente 1500 informações de proposições pudessem ser baixadas, então o scraper possibilitou que todas as informações fossem coletadas sem limitações. A coleta dos dados deste projeto foi feita no dia 24/01/2021.</p>
<p> O HTML do site foi analisado e utilizado para automatizar o maior número de processos possíveis. O Código preenche a pesquisa avançada automaticamente, checa quantas informações precisam ser baixadas e descobre a partir do número de proposições por página quando precisa parar de rodar. No fim salva todas informações coletadas em um arquivo excel</p>

In [1]:
# Importa bibliotecas
from selenium import webdriver
import pandas as pd
import time

In [13]:
# Informações da pesquisa avançada

# Chaves de pesquisa
chaves = ['Saúde pública','Sistema Único de Saúde']

# Janela de datas de apresentação a se procurar
data_inicial = '01012010'
data_final = '01072020'

# Botões a se apertar no site (PLP, PEC e PL)
botoes = ['PLP\ \ \ \ \ \ \ \ -\ Projeto\ de\ Lei\ Complementar',
          'PEC\ \ \ \ \ \ \ \ -\ Proposta\ de\ Emenda\ à\ Constituição',
          'PL\ \ \ \ \ \ \ \ \ -\ Projeto\ de\ Lei']

## Raspando informações - Chave: Saúde pública

In [21]:
# Abre o google crhome
driver = webdriver.Chrome()
try:
    driver.get('https://www.camara.leg.br/buscaProposicoesWeb/pesquisaAvancada')
    ## ------ início pesquisa avançada ------------
    # Aperta oos botões baseado no ID deles
    for bot in botoes:
        botao = driver.find_element_by_id(bot)
        botao.click()
    # Seleciona a checkbox sim para Em Tramitação
    is_tram_yes = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div/div/div[2]/div/div/div/div/form/fieldset[2]/fieldset/ul/li[2]/input')
    is_tram_yes.click()
    
    #time.sleep(2)
    # Encontra os campos de datas
    DataInicial = driver.find_element_by_id('dataInicialApresentacao')
    DataFinal = driver.find_element_by_id('dataFinalApresentacao')
    # Preenche os campos de data
    DataInicial.send_keys(data_inicial)
    DataFinal.send_keys(data_final)
    
    #time.sleep(3)
    # Encontra e clica no checkbox inteiro teor
    inteiroteor = driver.find_element_by_id('inteiroteor')
    inteiroteor.click()
    #time.sleep(2)
    # Encontra o local do input de palavras chaves e escreve a palavra
    input_palavra_chave = driver.find_element_by_id('todasaspalavras')
    input_palavra_chave.send_keys(chaves[0])
    #time.sleep(2)
    # Encontra e aperta o botão de pesquisar
    submit_butt = driver.find_element_by_id('pesquisar')
    submit_butt.click()
    ## ------ fim pesquisa avançada ------------
    ## ------ início do scraping das informações ---------- 
    PLs = []
    Autores = []
    Datas = []
    # Passa por todas páginas e salva as informações importantes (Proposição, Autor e data de apresentação)
    while True:
        numeros_PL = driver.find_elements_by_class_name('titulo')
        for PL in numeros_PL:
            nome = PL.find_element_by_tag_name('span')
            PLs.append(nome.get_attribute("innerHTML"))

        autor_PL = driver.find_elements_by_class_name('autor')
        for autor in autor_PL:
            nome = autor.find_element_by_tag_name('span')
            Autores.append(nome.get_attribute("innerHTML"))

        data_PL = driver.find_elements_by_class_name('dataApres')
        for data in data_PL:
            nome = data.find_element_by_tag_name('span')
            nome = nome.find_element_by_tag_name('span')
            Datas.append(nome.get_attribute("innerHTML").replace('&nbsp;',''))

        resultado = driver.find_element_by_id('msgQtdResultados')
        resultado = resultado.find_elements_by_tag_name('strong')

        final = resultado[2].get_attribute("innerHTML")
        ref = resultado[1].get_attribute("innerHTML")
        print(ref)
        if final == ref:
            break

        next_button = driver.find_element_by_link_text('Próxima')
        next_button.click()
        ## ------ fim do scraping das informações ---------- 
except Exception as e:
    print(e)
finally:
    # Fecha o google crhome
    driver.close()

20
40
60
80
100
120
120
120
120
120
140
160
180
200
220
240
260
280
280
280
280
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
680
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2300
2320
2340
2360
2380
2400
2420
2440
2460
2480
2500
2520
2540
2560
2580
2600
2620
2640
2660
2680
2700
2720
2740
2760
2780
2800
2820
2840
2860
2880
2900
2920
2940
2960
2980
3000
3020
3040
3060
3080
3100
3120
3140
3160
3180
3200
3220
3240
3260
3280
3300
3320
3340
3360
3380
3394


In [22]:
# Salva informações em um excel
infos = pd.DataFrame({'PL':PLs,'Autor':Autores, 'Data':Datas})
infos.drop_duplicates().to_excel('saude_publica.xlsx')

## Raspando informações - Chave: Sistema Único de Saúde

In [27]:
# Abre o google crhome
driver = webdriver.Chrome()
try:
    driver.get('https://www.camara.leg.br/buscaProposicoesWeb/pesquisaAvancada')
    ## ------ início pesquisa avançada ------------
    # Aperta oos botões baseado no ID deles
    for bot in botoes:
        botao = driver.find_element_by_id(bot)
        botao.click()
    # Seleciona a checkbox sim para Em Tramitação
    is_tram_yes = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div/div/div[2]/div/div/div/div/form/fieldset[2]/fieldset/ul/li[2]/input')
    is_tram_yes.click()
    
    #time.sleep(2)
    # Encontra os campos de datas
    DataInicial = driver.find_element_by_id('dataInicialApresentacao')
    DataFinal = driver.find_element_by_id('dataFinalApresentacao')
    # Preenche os campos de data
    DataInicial.send_keys(data_inicial)
    DataFinal.send_keys(data_final)
    
    #time.sleep(3)
    # Encontra e clica no checkbox inteiro teor
    inteiroteor = driver.find_element_by_id('inteiroteor')
    inteiroteor.click()
    #time.sleep(2)
    # Encontra o local do input de palavras chaves e escreve a palavra
    input_palavra_chave = driver.find_element_by_id('todasaspalavras')
    input_palavra_chave.send_keys(chaves[1])
    #time.sleep(2)
    # Encontra e aperta o botão de pesquisar
    submit_butt = driver.find_element_by_id('pesquisar')
    submit_butt.click()
    ## ------ fim pesquisa avançada ------------
    ## ------ início do scraping das informações ---------- 
    PLs = []
    Autores = []
    Datas = []
    # Passa por todas páginas e salva as informações importantes (Proposição, Autor e data de apresentação)
    while True:
        numeros_PL = driver.find_elements_by_class_name('titulo')
        for PL in numeros_PL:
            nome = PL.find_element_by_tag_name('span')
            PLs.append(nome.get_attribute("innerHTML"))

        autor_PL = driver.find_elements_by_class_name('autor')
        for autor in autor_PL:
            nome = autor.find_element_by_tag_name('span')
            Autores.append(nome.get_attribute("innerHTML"))

        data_PL = driver.find_elements_by_class_name('dataApres')
        for data in data_PL:
            nome = data.find_element_by_tag_name('span')
            nome = nome.find_element_by_tag_name('span')
            Datas.append(nome.get_attribute("innerHTML").replace('&nbsp;',''))
        
        # Encontra quantas proposições já foram baixadas e quantas faltam
        resultado = driver.find_element_by_id('msgQtdResultados')
        resultado = resultado.find_elements_by_tag_name('strong')
        final = resultado[2].get_attribute("innerHTML")
        ref = resultado[1].get_attribute("innerHTML")
        
        print(f'{ref}:{final}')
        # Checa se está na última página, caso sim, termina o código
        if final == ref:
            break
        
        # Clica no botão de Next
        next_button = driver.find_element_by_link_text('Próxima')
        next_button.click()
        ## ------ fim do scraping das informações ---------- 
except Exception as e:
    print(e)
finally:
    # Fecha o google crhome
    driver.close()

20:1610
40:1610
60:1610
80:1610
100:1610
100:1610
100:1610
100:1610
100:1610
100:1610
120:1610
140:1610
160:1610
180:1610
200:1610
220:1610
240:1610
260:1610
280:1610
300:1610
320:1610
340:1610
360:1610
380:1610
400:1610
420:1610
440:1610
460:1610
480:1610
500:1610
520:1610
540:1610
560:1610
580:1610
600:1610
620:1610
640:1610
660:1610
680:1610
700:1610
720:1610
740:1610
760:1610
780:1610
800:1610
820:1610
840:1610
860:1610
880:1610
900:1610
920:1610
940:1610
960:1610
980:1610
1000:1610
1020:1610
1040:1610
1060:1610
1080:1610
1100:1610
1120:1610
1140:1610
1160:1610
1180:1610
1200:1610
1220:1610
1240:1610
1260:1610
1280:1610
1300:1610
1320:1610
1340:1610
1360:1610
1380:1610
1400:1610
1420:1610
1440:1610
1460:1610
1480:1610
1500:1610
1520:1610
1540:1610
1560:1610
1580:1610
1600:1610
1610:1610


In [28]:
# Salva informações em um excel
infos2 = pd.DataFrame({'PL':PLs2,'Autor':Autores2, 'Data':Datas2})
infos2.drop_duplicates().to_excel('SUS.xlsx')